In [1]:
from openmm.app import ForceField, Modeller, PDBFile
import openmm
import json
from cosolvkit.parametrize import _parametrize_cosolvents
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')


In [2]:
small_molecule_ff = _parametrize_cosolvents(cosolvents=["c1ccccc1"], small_molecule_ff="espaloma")
forcefield.registerTemplateGenerator(small_molecule_ff.generator) 

string indices must be integers, not 'str'
c1ccccc1


In [7]:
from openmm import Vec3
from openmm.app import Topology
from openmm.unit import nanometer, norm, angstrom
import openmm.unit as unit
import numpy as np
from scipy import spatial

def create_box(positions, padding, radius=None):
    padding = padding.value_in_unit(nanometer)
    if positions is not None:
        positions = positions.value_in_unit(nanometer)
        minRange = Vec3(*(min((pos[i] for pos in positions)) for i in range(3)))
        maxRange = Vec3(*(max((pos[i] for pos in positions)) for i in range(3)))
        center = 0.5*(minRange+maxRange)
        radius = max(unit.norm(center-pos) for pos in positions)
        print(radius)
    else:
        radius = radius.value_in_unit(nanometer)
    width = max(2*radius+padding, 2*padding)
    vectors = (Vec3(width, 0, 0), Vec3(0, width, 0), Vec3(0, 0, width))
    box = Vec3(vectors[0][0], vectors[1][1], vectors[2][2])
    return vectors, box

def add_cosolvent(cosolvent, concentration, receptor=None, modeller=None, padding=None, radius=None):
    distance_from_edges = 3.
    distance_from_receptor = 4.5
    distance_from_cosolvent = 2.5
    
    if receptor is None and (padding is None and radius is None):
        print("Error, if no receptor available padding and radius need to be passed manually.")

    if receptor is None:
        vectors, box = create_box(positions=None, padding=padding, radius=radius)
        new_topology = Topology()
        new_topology.setPeriodicBoxVectors(vectors*nanometer)
        center = Vec3(0, 0, 0)
    else:
        vectors = modeller.topology.getPeriodicBoxVectors()
        new_topology = modeller.topology
        center = [(max((pos[i] for pos in modeller.positions))+min((pos[i] for pos in modeller.positions)))/2 for i in range(3)]
        center = Vec3(center[0], center[1], center[2])
        pdb_topology = receptor.getTopology()
        pdb_positions = receptor.getPositions().value_in_unit(nanometer)
        pdb_residues = list(pdb_topology.residues())
        # pdb_box_size = pdb_topology.getUnitCellDimensions().value_in_unit(nanometer)

                    
    # cosolvent_xyz = cosolvent.positions
    modeller.topology = new_topology
    modeller.positions = modeller.addPositions(new_positions)
    return vectors

In [8]:
pdb = PDBFile("protein_clean.pdb")
modeller = Modeller(pdb.topology, pdb.positions)
if modeller.getTopology() is not None:
    modeller.deleteWater()
vectors, box = create_box(modeller.positions, 12*angstrom)
modeller.topology.setPeriodicBoxVectors(vectors)

from cosolvkit.cosolvent import CoSolvent
with open("benzene.json") as fi:
    c = json.load(fi)
cosolvent = CoSolvent(**c)

# add_cosolvent(cosolvent, None, pdb, modeller, 12*angstrom, None)
# modeller.addSolvent()

4.636437847367309


In [22]:
from cosolvkit.cosolvent import CoSolvent
with open("benzene.json") as fi:
    c = json.load(fi)
cosolvent = CoSolvent(**c)

AttributeError: 'CoSolvent' object has no attribute 'topology'

In [31]:
cos_top = Topology()
newAtoms = {}
newChain = cos_top.addChain()
newRes = cos_top.addResidue(cosolvent.resname, newChain)
molAtoms = []
for atom in cosolvent.atom_names:
    molAtoms.append(cos_top.addAtom(atom, atom, newRes))

for bond_idx in range(len(cosolvent.pdb_conect)):
    for b2 in cosolvent.pdb_conect[bond_idx]:
        cos_top.addBond(bond_idx, b2)

In [32]:
modeller = Modeller(cos_top, cosolvent.positions)
PDBFile.writeFile(
            modeller.topology,
            modeller.positions,
            open("benzene_clean.pdb",
                    "w"),
            keepIds=True)


FileNotFoundError: [Errno 2] No such file or directory: 'benzene_clean.pdb'